<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/Labs/deepNN_lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tomato Leaf Disease Prediction  

_(Using Alex Net)_

## Initial Set-Up

In [1]:
## we'll work with and hence always remember to connect to the gpu runtime before starting the execution

In [2]:
!nvidia-smi

Tue Jun  8 07:59:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install tensorflow-gpu

     |████████████████████████████████| 454.3MB 36kB/s 


## Data

In [8]:
## required libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils as utils
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3 ## check it's arch, a 42 layer layer net
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from glob import glob

import numpy as np
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
import seaborn 
import cv2
import IPython
from six.moves import urllib
print(tf.__version__)

from google.colab import drive

2.5.0


In [4]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
dataDir = 'drive/MyDrive/git/Datasets/tomato/'

In [10]:
## see if we are at right loc
!ls drive/MyDrive/git/Datasets/tomato/

train  valid


## work

In [12]:
## will need to resize all images (if ot so) 
IMAGE_SIZE = [224,224]

In [27]:
train_path = dataDir+'train'
valid_path = dataDir+'valid'

_importing the inception-v3 lib. & adding preprocessing layer to the front of inception v3_  

_here it's done using imagenet weights_

In [16]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [17]:
# don't traing existing weights
for layer in inception.layers:
  layer.trainable = False

In [18]:
## for getting number of unique output class
folders = glob(train_path + '/*')

In [20]:
len(folders), folders

(10,
 ['drive/MyDrive/git/Datasets/tomato/train/Tomato___Leaf_Mold',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Tomato_mosaic_virus',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Early_blight',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Target_Spot',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Late_blight',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___healthy',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Bacterial_spot',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Septoria_leaf_spot',
  'drive/MyDrive/git/Datasets/tomato/train/Tomato___Spider_mites Two-spotted_spider_mite'])

_so, we have 10 classes_

In [21]:
# adding layers
x = Flatten()(inception.output)

In [23]:
prediction = Dense(len(folders), activation='softmax')(x)

## creating a model object
model = Model(inputs=inception.input, outputs=prediction)

In [24]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 111, 111, 32) 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_94[0][0]     
______________________________________________________________________________________________

In [25]:
model.compile(
    loss='categorica_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [28]:
## using the imgages data generator 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [29]:
## training  ---  providing same target size as that for image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 18355 images belonging to 10 classes.


In [31]:
test_set = test_datagen.flow_from_directory(valid_path,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 4585 images belonging to 10 classes.


In [39]:
## fit the model - a time consuming process
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
    )

Epoch 1/10


ValueError: ignored